In [20]:
from numpy import loadtxt
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib
import h5py
from numpy import expand_dims
import numpy as np
matplotlib.use("Agg")
import sys, os, numpy
import tensorflow
from qkeras.utils import load_qmodel
import argparse

2024-02-05 15:04:36.370254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


    I am making this work by creating dictionaries (variable ={}). Thus, the keyword for a respective dataset calls specific paths or objects. To see what's in the dictionary, just type its name and run the cell. Otherwise, `dictionary.keys()` also works, but it might not show subkeys.

In [6]:
dataLocations4c = {"original":"newTestDataST30.h5", 
                   "M_LLP_15_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_15_ctau_100/newTestDataPT20_Signal_Only.h5", 
                   "M_LLP_30_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_30_ctau_100/newTestDataPT20_Signal_Only.h5", 
                   "M_LLP_60_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_60_ctau_100/newTestDataPT20_Signal_Only.h5"
                   }

In [7]:
dataLocations4c["original"]

'newTestDataST30.h5'

In [5]:
len(dataLocations)

4

In [45]:
modelLocations4c = {"original":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/qkL1JetTagModel.h5",
                    "M_LLP_15_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_Scripts/M_LLP_15_ctau_100/M_LLP_15_ctau_100qkL1JetTagModel.h5", 
                    "M_LLP_30_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_Scripts/M_LLP_30_ctau_100/M_LLP_30_ctau_100qkL1JetTagModel.h5", 
                    "M_LLP_60_ctau_100":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_Scripts/M_LLP_60_ctau_100/M_LLP_60_ctau_100qkL1JetTagModel.h5" }

In [7]:
len(modelLocations)

4

In [44]:
keys4c = ["original", "M_LLP_15_ctau_100", "M_LLP_30_ctau_100", "M_LLP_60_ctau_100"]

    Only one sample of 4b tested so far.

In [10]:
dataLocations4b = {
    "original":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/qkL1JetTagModel.h5",
    "M_LLP_30_ctau_10":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/4b/M_LLP_30_ctau_10/newTestDataPT20_Signal_Only.h5"
    }

In [43]:
modelLocations4b = {
                    "original":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/qkL1JetTagModel.h5",
                    "M_LLP_30_ctau_10":"/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_Scripts/4b/M_LLP_30_ctau_10/qkL1JetTagModel.h5"
                    }

In [8]:
keys4b = ["original","M_LLP_30_ctau_10"]

    Below I start the plotting. Do not change the bkgTestFile. This has the testing data with QCD jets we've been using.

In [18]:
N_PART_PER_JET = 10
N_FEAT = 14
bkgTestFile = "/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_30_ctau_100/testingDataQCD30.h5"

In [41]:
results = {}

In [60]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

    The loop below iterates over the 4c samples by the keywords stored in the list keys4c. For 4b, make a new list `keys4b` with desired keys and change  4b to 4c everywhere in loop below.

In [61]:
for keys in keys4c:
    print(dataLocations4c[keys])
    with h5py.File(dataLocations4c[keys], "r") as hf:
        dataset = hf["Testing Data"][:]
    with h5py.File(bkgTestFile, "r") as hf:
        datasetQCD = hf["Testing Data"][:]

    dataset = np.concatenate((dataset,datasetQCD)) #Stacking datasets on top of another
    np.random.shuffle(dataset) #shuffling rows

    A = dataset[:, 0 : len(dataset[0]) - 1]
    b = dataset[:, len(dataset[0]) - 1]
    #A = expand_dims(A, axis=3)
    A = A.reshape((A.shape[0], N_PART_PER_JET, N_FEAT))

    model = load_qmodel(modelLocations4c[keys])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["binary_accuracy"])

    y_qkeras = model.predict(A).ravel()

    fpr, tpr, thresholds_Ab = roc_curve(b, y_qkeras)
    auc_val = auc(fpr, tpr) 
    results[keys] = {"tpr": tpr, "fpr": fpr, "auc": auc_val }   
    

newTestDataST30.h5


3409/3409 [==============================] - 9s 3ms/step
/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_15_ctau_100/newTestDataPT20_Signal_Only.h5


3967/3967 [==============================] - 10s 2ms/step
/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_30_ctau_100/newTestDataPT20_Signal_Only.h5


3781/3781 [==============================] - 9s 2ms/step
/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/02-02_datasets/M_LLP_60_ctau_100/newTestDataPT20_Signal_Only.h5


3404/3404 [==============================] - 9s 2ms/step


In [62]:
import matplotlib.pyplot as plt

    Dictionary containing results is below.

In [65]:
results

{'original': {'tpr': array([0.        , 0.        , 0.        , ..., 0.99300019, 0.99309741,
         1.        ]),
  'fpr': array([0.00000000e+00, 1.01241217e-05, 3.03723652e-05, ...,
         8.34338996e-01, 8.38246907e-01, 1.00000000e+00]),
  'auc': 0.9417839968647547},
 'M_LLP_15_ctau_100': {'tpr': array([0.00000000e+00, 3.55113636e-05, 7.10227273e-05, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
  'fpr': array([0.00000000e+00, 0.00000000e+00, 2.02482435e-05, ...,
         9.99878511e-01, 9.99959504e-01, 1.00000000e+00]),
  'auc': 0.9359682514804802},
 'M_LLP_30_ctau_100': {'tpr': array([0.00000000e+00, 4.50551926e-05, 9.01103852e-05, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
  'fpr': array([0.        , 0.        , 0.        , ..., 0.99993926, 0.9999595 ,
         1.        ]),
  'auc': 0.9278922997617837},
 'M_LLP_60_ctau_100': {'tpr': array([0.00000000e+00, 9.85221675e-05, 1.08374384e-03, ...,
         1.00000000e+00, 1.00000000e+00, 1.000

    Below I make the plots for 4c samples since we are iterating over those paths above. Note that I am making them by calling respective keys individually, i.e., doing `plt.plot` for each sample.

In [84]:
#plt.plot(fpr_Ab, tpr_Ab, label="Pf, AUC={:.3f}".format(auc_Ab))
plt.figure()
plt.plot(results["original"]["fpr"], results["original"]["tpr"], 
         label="stop,AUC={:.3f}".format(results["original"]["auc"]))
plt.plot(results["M_LLP_15_ctau_100"]["fpr"], results["M_LLP_15_ctau_100"]["tpr"], 
         label="4c_M_LLP_15_ctau_100,AUC={:.3f}".format(results["M_LLP_15_ctau_100"]["auc"]))
plt.plot(results["M_LLP_30_ctau_100"]["fpr"], results["M_LLP_30_ctau_100"]["tpr"], 
         label="4c_M_LLP_30_ctau_100,AUC={:.3f}".format(results["M_LLP_30_ctau_100"]["auc"]))
plt.plot(results["M_LLP_60_ctau_100"]["fpr"], results["M_LLP_60_ctau_100"]["tpr"], 
         label="4c_M_LLP_60_ctau_100,AUC={:.3f}".format(results["M_LLP_60_ctau_100"]["auc"]))

plt.xlabel("Background Efficiency", fontsize=16)
plt.ylabel("Signal Efficiency", fontsize=16)

plt.title("L1 LLP Tag QK Model ROC Curve", fontsize=16, weight="bold")
plt.legend(loc="best")
plt.xscale("log")
plt.grid(True)

plt.savefig("ALLrocCurveQK.pdf", dpi=120)